## import the librarys

getting an accuracy of 90% 

In [ ]:
import numpy as np
import pandas as pd

import keras,os
from keras.layers import Conv2D, MaxPool2D,Dropout,Flatten,Dense
from keras.preprocessing import image 
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3

##import the data

In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
train = train_data_gen.flow_from_directory(directory="/content/drive/MyDrive/x-ray data/train" , target_size=(256,256) , batch_size=32, class_mode = 'binary')

Found 4852 images belonging to 2 classes.


In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
test = train_data_gen.flow_from_directory(directory="/content/drive/MyDrive/x-ray data/test" , target_size=(256,256) , batch_size=32,  class_mode = 'binary')

Found 240 images belonging to 2 classes.


In [ ]:
train.class_indices

{'COVID': 0, 'Normal': 1}

##inception( Transfer learning)

In [ ]:
Inception = InceptionV3(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')

In [ ]:
# Inception.trainable=False
#for layer in Inception.layers:
Inception.trainable = False

# now 13 layers are not trainable
# to make last 3 dense trainable layers
model=keras.Sequential([
    Inception,
    keras.layers.Flatten(), #flattern the above data
    keras.layers.Dense(units=1024, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=1, activation="sigmoid"),
])

model.compile(optimizer="adam", loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

##check the model

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 flatten_4 (Flatten)         (None, 73728)             0         
                                                                 
 dense_8 (Dense)             (None, 1024)              75498496  
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_9 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 97,302,305
Trainable params: 75,499,521
Non-trainable params: 21,802,784
_________________________________________________________________


##train the data

In [ ]:
model.fit_generator(train, steps_per_epoch=10,epochs=10,validation_steps=8, validation_data=test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
10/10 [==============================] - 96s 9s/step - loss: 29.4938 - accuracy: 0.6031 - val_loss: 12.2790 - val_accuracy: 0.6292
Epoch 2/10
10/10 [==============================] - 88s 9s/step - loss: 7.9069 - accuracy: 0.7219 - val_loss: 10.0731 - val_accuracy: 0.6792
Epoch 3/10
10/10 [==============================] - 88s 9s/step - loss: 5.4472 - accuracy: 0.7937 - val_loss: 6.2176 - val_accuracy: 0.7250
Epoch 4/10
10/10 [==============================] - 88s 9s/step - loss: 3.9813 - accuracy: 0.8062 - val_loss: 2.3738 - val_accuracy: 0.8708
Epoch 5/10
10/10 [==============================] - 86s 9s/step - loss: 2.9888 - accuracy: 0.8313 - val_loss: 2.0401 - val_accuracy: 0.8708
Epoch 6/10
10/10 [==============================] - 87s 9s/step - loss: 1.5104 - accuracy: 0.8625 - val_loss: 1.2951 - val_accuracy: 0.8875
Epoch 7/10
10/10 [==============================] - 86s 9s/step - loss: 1.1431 - accuracy: 0.8906 - val_loss: 0.8932 - val_accuracy: 0.9000
Epoch 8/10
10/10 

## random image

non covid random image

In [ ]:
path= "/content/normal.jpeg"
img=image.load_img(path,target_size=(256,256))
img= image.img_to_array(img)/255
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
model.predict(img)

array([[1.]], dtype=float32)

covid image

In [ ]:
path= "/content/COVID-3322.png"
img=image.load_img(path,target_size=(256,256))
img= image.img_to_array(img)/255
img=np.array([img])
img.shape

FileNotFoundError: ignored

In [ ]:
model.predict(img)

array([[0.99888986, 0.0011101 ]], dtype=float32)